# 安装google云盘必要的包

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
!nvidia-smi

Fri Mar 20 10:53:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144542 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.18-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

# 连接google云盘

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!ls drive/kaikeba/Abstract/model/pgn

layers.py  pgn_model.py  __pycache__


In [0]:
!cat drive/kaikeba/Abstract/model/pgn/pgn_model.py

# 载入代码

In [0]:
! pip install rouge

In [0]:
! pip install tensorflow-gpu==2.0.0

  Using cached https://files.pythonhosted.org/packages/fc/08/8b927337b7019c374719145d1dceba21a8bb909b93b1ad6f8fb7d22c1ca1/tensorflow_estimator-2.0.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/76/54/99b9d5d52d5cb732f099baaaf7740403e83fe6b0cedde940fabd2b13d75a/tensorboard-2.0.2-py3-none-any.whl
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensor

In [0]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [0]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('drive/kaikeba/Abstract/')

# 导入训练代码

In [0]:
from utils.textprocessing import load_embedding_matrix
from utils.data_loader import load_data
from model.pgn.pgn_model import AttentionModel
import pandas as pd

# 预处理数据

In [0]:
# build_dataset(train_data_path, test_data_path)

# 参数设置

In [0]:
!nvidia-smi

Wed Mar 18 17:45:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    27W /  70W |   8811MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
embedding_matrix, word_index_dict = load_embedding_matrix()
# train_x, train_y, test_x = load_data()

In [0]:
train_x = pd.read_csv('drive/kaikeba/Abstract/data/train_X_seg_data.csv', encoding='utf-8', squeeze=True, header=None)
with open('drive/kaikeba/Abstract/data/train_Y_seg_data.csv', 'r') as file:
  content = file.read()
  train_y = content.splitlines()
# train_y = pd.read_csv('drive/kaikeba/Abstract/data/train_Y_seg_data.csv', encoding='utf-8', squeeze=True, header=None)

In [0]:
temp = []
for y in train_y:
  y_list = y.split()
  temp_str = '<start> ' + ' '.join(y_list[:min(32, len(y_list))]) +' <end>'
  temp.append(temp_str)
train_y = pd.Series(temp) 

# 训练模型

In [0]:
# GPU 700s TPU x s
x = train_x
y = train_y
model = AttentionModel(embedding_matrix=embedding_matrix, word_index_dict=word_index_dict, max_length_input=200, max_length_output=34, batch_size=32, epochs=4, encoder_units=256, attention_units=256, decoder_units=256, learning_rate=0.15)
checkpoint = tf.train.Checkpoint(PNG=model)
checkpoint_manager = tf.train.CheckpointManager(checkpoint, 'drive/kaikeba/Abstract/data/checkpoints/test2', max_to_keep=5)
# checkpoint_manager.save()
checkpoint.restore(checkpoint_manager.latest_checkpoint)
model.fit(x, y)
checkpoint = tf.train.Checkpoint(PNG=model)
checkpoint_manager = tf.train.CheckpointManager(checkpoint, 'drive/kaikeba/Abstract/data/checkpoints/test2', max_to_keep=5)
checkpoint_manager.save()

Streaming output truncated to the last 5000 lines.
Epoch 3 Batch 176 Loss 2.1413 Log Loss 2.0350 Coverage Loss 0.1064 Time 0.6807670593261719 sec
Epoch 3 Batch 177 Loss 2.0749 Log Loss 1.9702 Coverage Loss 0.1047 Time 0.6621603965759277 sec
Epoch 3 Batch 178 Loss 1.8419 Log Loss 1.7670 Coverage Loss 0.0749 Time 0.6520297527313232 sec
Epoch 3 Batch 179 Loss 2.3743 Log Loss 2.2766 Coverage Loss 0.0977 Time 0.7595260143280029 sec
Epoch 3 Batch 180 Loss 2.1060 Log Loss 2.0103 Coverage Loss 0.0957 Time 0.6623179912567139 sec
Epoch 3 Batch 181 Loss 1.7404 Log Loss 1.6658 Coverage Loss 0.0746 Time 0.6572098731994629 sec
Epoch 3 Batch 182 Loss 1.6739 Log Loss 1.6200 Coverage Loss 0.0539 Time 0.665574312210083 sec
Epoch 3 Batch 183 Loss 2.0580 Log Loss 1.9776 Coverage Loss 0.0803 Time 0.6623709201812744 sec
Epoch 3 Batch 184 Loss 2.0813 Log Loss 2.0015 Coverage Loss 0.0799 Time 0.6542847156524658 sec
Epoch 3 Batch 185 Loss 2.3817 Log Loss 2.2782 Coverage Loss 0.1035 Time 0.7745778560638428 sec


'drive/kaikeba/Abstract/data/checkpoints/test2/ckpt-1'

# 生成结果

In [0]:
def submit_proc(sentence):
    sentence = sentence.replace('<start>', '').replace('<end>', '').replace('<pad>', '')
    sentence=sentence.lstrip(' ，！。')
    sentence=sentence.replace(' ','')
    if sentence=='':
        sentence='随时联系'
    return sentence

In [0]:
model = AttentionModel(embedding_matrix=embedding_matrix, word_index_dict=word_index_dict, max_length_input=200, max_length_output=34, batch_size=32, epochs=10, encoder_units=256, attention_units=256, decoder_units=256, learning_rate=0.15)
checkpoint = tf.train.Checkpoint(PNG=model)
checkpoint_manager = tf.train.CheckpointManager(checkpoint, 'drive/kaikeba/Abstract/data/checkpoints/test2', max_to_keep=5)
# checkpoint_manager.save()
checkpoint.restore(checkpoint_manager.latest_checkpoint)

In [0]:
test_x = pd.read_csv('drive/kaikeba/Abstract/data/test_X_seg_data.csv', encoding='utf-8', squeeze=True, header=None)

In [0]:
test_y = []
for i in range(len(test_x)):
  print(i)
  test_y.append(model.beam_predict(test_x[i: i + 1])[0])
test_y = [submit_proc(item) for item in test_y]

Streaming output truncated to the last 5000 lines.
14580
14581
14582
14583
14584
14585
14586
14587
14588
14589
14590
14591
14592
14593
14594
14595
14596
14597
14598
14599
14600
14601
14602
14603
14604
14605
14606
14607
14608
14609
14610
14611
14612
14613
14614
14615
14616
14617
14618
14619
14620
14621
14622
14623
14624
14625
14626
14627
14628
14629
14630
14631
14632
14633
14634
14635
14636
14637
14638
14639
14640
14641
14642
14643
14644
14645
14646
14647
14648
14649
14650
14651
14652
14653
14654
14655
14656
14657
14658
14659
14660
14661
14662
14663
14664
14665
14666
14667
14668
14669
14670
14671
14672
14673
14674
14675
14676
14677
14678
14679
14680
14681
14682
14683
14684
14685
14686
14687
14688
14689
14690
14691
14692
14693
14694
14695
14696
14697
14698
14699
14700
14701
14702
14703
14704
14705
14706
14707
14708
14709
14710
14711
14712
14713
14714
14715
14716
14717
14718
14719
14720
14721
14722
14723
14724
14725
14726
14727
14728
14729
14730
14731
14732
14733
14734
14735
14736
14737
1

In [0]:
idx_list = []
for i in range(len(test_y)):
  idx_list.append('Q{}'.format(i + 1))
result_df = pd.DataFrame()
result_df['QID'] = pd.Series(idx_list)
result_df['Prediction'] = pd.Series(test_y)
result_df.to_csv('drive/kaikeba/Abstract/data/result_scheduled_sampling.csv', encoding='utf-8', index=False)